Loading datasets

In [1]:
# to be completed
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [2]:
%cd /content/drive/'My Drive'/'IEMOCAP_full_release_withoutVideos_sentenceOnly'/'IEMOCAP_full_release'

/content/drive/.shortcut-targets-by-id/1H_wIpsIF42ILnS8ZNz7BovI5ga272ikl/IEMOCAP_full_release_withoutVideos_sentenceOnly/IEMOCAP_full_release


In [3]:
!pwd

/content/drive/.shortcut-targets-by-id/1H_wIpsIF42ILnS8ZNz7BovI5ga272ikl/IEMOCAP_full_release_withoutVideos_sentenceOnly/IEMOCAP_full_release


Importing libraries

In [4]:
!pip install tensorboardX

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 2.3 MB/s eta 0:00:00


In [5]:
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.backend_bases import RendererBase
from scipy import signal
from scipy.io import wavfile
import os
from PIL import Image
from scipy.fftpack import fft
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils import data
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from tensorboardX import SummaryWriter

Preprocessing the data

In [6]:
import numpy as np
import os
import pandas
import librosa

In [7]:
def get_labels(annot_file, file_name):

    f = open(annot_file, 'r').read()
    f = f.split('\n')
    f = f[2:]

    for data in f:

        if len(data) > 0:
            if data[0] == '[':
                data2 = data.split('\t')

                if data2[1] == file_name:
                    emo = data2[2]
                    vad = data2[3][1:-1].split(', ')
                    return emo, [float(x) for x in vad]

    raise ValueError('Label not found')

def get_mocap_rot(path):

    f = open(path, 'r').read()
    f = np.array(f.split('\n'))
    header = f[0].split(' ')
    xyz = f[1].split(' ')
    f = f[2:]
    data_list = []

    for data in f:
        data2 = data.split(' ')
        if(len(data2)<2):
            continue
        dic = {'frame': data2[0], 'time': data2[1],
               'markers': np.array(data2[2:]).astype(float)}
        data_list.append(dic)

    return header, xyz, data_list

def get_wav(path):
    x, sr = librosa.load(path, sr=16000)
    return x, sr

def get_ph_fa(path):
    f = open(path, 'r').read()
    f = np.array(f.split('\n'))
    header = f[0].split()
    f = f[1:-2]
    data_list = []

    for data in f:
        data2 = data.split()
        dic = {'SFrm':np.array(data2[0]).astype(int),
               'EFrm':np.array(data2[1]).astype(int),
               'SegAScr':np.array(data2[2]).astype(int),
               'Phone':data2[3]}
        data_list.append(dic)

    return header, data_list

def frame_to_s(fr):
    return (fr+2)*10/1000

In [8]:
root_path = '/content/drive/.shortcut-targets-by-id/1H_wIpsIF42ILnS8ZNz7BovI5ga272ikl/IEMOCAP_full_release_withoutVideos_sentenceOnly'

df = pandas.read_csv(os.path.join(root_path, 'iemocap.csv'))
sessions = [1, 2, 3, 4, 5]
df = df[df['session'].isin(sessions)]

# Remove unwanted emotions and empty values
unwanted_emotions = ['xxx', '', 'oth', 'dis', 'sur', 'fea']
df = df[~df['emotion'].isin(unwanted_emotions)]

# Calculate annotator difference
df['annotator_difference'] = df['n_annotators'] - df['agreement']

# Filter by annotator difference
df = df[df['annotator_difference'] <= 1]

# Replace 'exc' emotion with 'hap'
df.loc[df['emotion'] == 'exc', 'emotion'] = 'hap'


emotions_count_before = df['emotion'].value_counts()
print("Emotions count before filtering:")
print(emotions_count_before)

# Group by emotion and select first 550 rows of each group
df = df.groupby('emotion').head(550)

# Count the occurrences of each emotion after filtering
emotions_count_after = df['emotion'].value_counts()
print("\nEmotions count after filtering:")
print(emotions_count_after)

# Display the first 5 rows
display(df)
#Ntotal = 5531 utterances that are labeled with one of
#five categories: Happy, Angry, Neutral, Sad, Exited. This set is
#mapped down to four categories by merging Exited and Happy


Emotions count before filtering:
fru    1692
neu    1551
hap    1471
ang    1077
sad    1000
Name: emotion, dtype: int64

Emotions count after filtering:
neu    550
fru    550
ang    550
hap    550
sad    550
Name: emotion, dtype: int64


,Unnamed: 0,session,method,gender,n_annotators,agreement,emotion,valence,activation,dominance,wav_path,MOCAP_rotated_path,MOCAP_head_path,MOCAP_hand_path,FA_ph_path,FA_st_path,FA_sy_path,FA_wd_path,annotator_difference
0,0,1,script,F,3,3,neu,2.5,2.0,2.0,IEMOCAP_full_release/Session1/sentences/wav/Se...,IEMOCAP_full_release/Session1/sentences/MOCAP_...,IEMOCAP_full_release/Session1/sentences/MOCAP_...,IEMOCAP_full_release/Session1/sentences/MOCAP_...,IEMOCAP_full_release/Session1/sentences/Forced...,IEMOCAP_full_release/Session1/sentences/Forced...,IEMOCAP_full_release/Session1/sentences/Forced...,IEMOCAP_full_release/Session1/sentences/Forced...,0
1,1,1,script,F,3,2,fru,2.5,2.0,2.5,IEMOCAP_full_release/Session1/sentences/wav/Se...,IEMOCAP_full_release/Session1/sentences/MOCAP_...,IEMOCAP_full_release/Session1/sentences/MOCAP_...,IEMOCAP_full_release/Session1/sentences/MOCAP_...,IEMOCAP_full_release/Session1/sentences/Forced...,IEMOCAP_full_release/Session1/sentences/Forced...,IEMOCAP_full_release/Session1/sentences/Forced...,IEMOCAP_full_release/Session1/sentences/Forced...,1
4,4,1,script,F,3,2,neu,2.0,3.0,3.0,IEMOCAP_full_release/Session1/sentences/wav/Se...,IEMOCAP_full_release/Session1/sentences/MOCAP_...,IEMOCAP_full_release/Session1/sentences/MOCAP_...,IEMOCAP_full_release/Session1/sentences/MOCAP_...,IEMOCAP_full_release/Session1/sentences/Forced...,IEMOCAP_full_release/Session1/sentences/Forced...,IEMOCAP_full_release/Session1/sentences/Forced...,IEMOCAP_full_release/Session1/sentences/Forced...,1
6,6,1,script,F,3,2,ang,2.0,3.5,3.0,IEMOCAP_full_release/Session1/sentences/wav/Se...,IEMOCAP_full_release/Session1/sentences/MOCAP_...,IEMOCAP_full_release/Session1/sentences/MOCAP_...,IEMOCAP_full_release/Session1/sentences/MOCAP_...,IEMOCAP_full_release/Session1/sentences/Forced...,IEMOCAP_full_release/Session1/sentences/Forced...,IEMOCAP_full_release/Session1/sentences/Forced...,IEMOCAP_full_release/Session1/sentences/Forced...,1
7,7,1,script,F,3,2,ang,2.0,3.5,2.5,IEMOCAP_full_release/Session1/sentences/wav/Se...,IEMOCAP_full_release/Session1/sentences/MOCAP_...,IEMOCAP_full_release/Session1/sentences/MOCAP_...,IEMOCAP_full_release/Session1/sentences/MOCAP_...,IEMOCAP_full_release/Session1/sentences/Forced...,IEMOCAP_full_release/Session1/sentences/Forced...,IEMOCAP_full_release/Session1/sentences/Forced...,IEMOCAP_full_release/Session1/sentences/Forced...,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5350,5350,3,impro,M,4,3,ang,2.0,3.0,4.0,IEMOCAP_full_release/Session3/sentences/wav/Se...,IEMOCAP_full_release/Session3/sentences/MOCAP_...,IEMOCAP_full_release/Session3/sentences/MOCAP_...,IEMOCAP_full_release/Session3/sentences/MOCAP_...,IEMOCAP_full_release/Session3/sentences/Forced...,IEMOCAP_full_release/Session3/sentences/Forced...,IEMOCAP_full_release/Session3/sentences/Forced...,IEMOCAP_full_release/Session3/sentences/Forced...,1
5493,5493,3,script,M,3,3,ang,2.0,4.0,4.0,IEMOCAP_full_release/Session3/sentences/wav/Se...,IEMOCAP_full_release/Session3/sentences/MOCAP_...,IEMOCAP_full_release/Session3/sentences/MOCAP_...,IEMOCAP_full_release/Session3/sentences/MOCAP_...,IEMOCAP_full_release/Session3/sentences/Forced...,IEMOCAP_full_release/Session3/sentences/Forced...,IEMOCAP_full_release/Session3/sentences/Forced...,IEMOCAP_full_release/Session3/sentences/Forced...,0
5494,5494,3,script,M,3,2,ang,3.0,2.5,3.0,IEMOCAP_full_release/Session3/sentences/wav/Se...,IEMOCAP_full_release/Session3/sentences/MOCAP_...,IEMOCAP_full_release/Session3/sentences/MOCAP_...,IEMOCAP_full_release/Session3/sentences/MOCAP_...,IEMOCAP_full_release/Session3/sentences/Forced...,IEMOCAP_full_release/Session3/sentences/Forced...,IEMOCAP_full_release/Session3/sentences/Forced...,IEMOCAP_full_release/Session3/sentences/Forced...,1
5497,5497,3,script,M,3,2,ang,2.0,4.0,3.5,IEMOCAP_full_release/Session3/sentences/wav/Se...,IEMOCAP_full_release/Session3/sentences/MOCAP_...,IEMOCAP_full_release/Session3/sentences/MOCAP_.

Preparing training / testing /  eval datasets

In [47]:
print(len(df))

num_samples = len(df)
train_end = int(num_samples * 0.7)
test_end = int(num_samples * 0.85)

train_data = df.iloc[:train_end]  # Selecting rows up to train_end

#display(train_data)
test_data = df.iloc[train_end:test_end]  # Selecting rows from train_end to test_end

#display(test_data)
eval_data = df.iloc[test_end:]  # Selecting rows from test_end onwards

train_features = []
train_labels = []
test_features = []
test_labels = []



for index , row in train_data.iterrows():
    #print(index)
    #print(row)
    session = row['session']
    method = row['method']
    gender = row['gender']
    emotion = row['emotion']
    n_annot = row['emotion']
    agreement = row['agreement']
    wav_path = row['wav_path']

    _, file_name = os.path.split(wav_path)

    annot_file = os.path.join(root_path, 'IEMOCAP_full_release',
                              'Session'+str(session),
                              'dialog', 'EmoEvaluation',
                              file_name[:-9] + '.txt')

    # get labels
    emo, vad = get_labels(annot_file, file_name[:-4])
    print(vad)

    # get MOCAP data
    header_rot, xyz_rot, data_rot = get_mocap_rot(os.path.join(root_path, row['MOCAP_rotated_path']))

    for dic in data_rot:
      tensor = torch.tensor(dic['markers'])
      tensor = F.normalize(tensor, p=2, dim=-1)
      if torch.isnan(tensor).any():
        # Passer au tensor suivant si le tensor contient des NaN
        continue
      train_features.append(tensor)
      train_labels.append(emo)

    #tensors_list = [torch.tensor(dic['markers']) for dic in data_rot]

    #train_features.append(tensors_list[0])
    #train_labels.append(emo)

    # get audio data
    #x, sr = get_wav(os.path.join(root_path, row['wav_path']))

    # get transcription data
    #header_ph, data_ph = get_ph_fa(os.path.join(root_path, row['FA_ph_path']))



for index, row in test_data.iterrows():

    session = row['session']
    method = row['method']
    gender = row['gender']
    emotion = row['emotion']
    n_annot = row['emotion']
    agreement = row['agreement']
    wav_path = row['wav_path']

    _, file_name = os.path.split(wav_path)

    annot_file = os.path.join(root_path, 'IEMOCAP_full_release',
                              'Session'+str(session),
                              'dialog', 'EmoEvaluation',
                              file_name[:-9] + '.txt')

    # get labels
    emo, vad = get_labels(annot_file, file_name[:-4])
    print(vad)

    # get MOCAP data
    header_rot, xyz_rot, data_rot = get_mocap_rot(os.path.join(root_path, row['MOCAP_rotated_path']))
    for dic in data_rot:
      tensor = torch.tensor(dic['markers'])
      tensor = F.normalize(tensor, p=2, dim=-1)
      if torch.isnan(tensor).any():
        # Passer au tensor suivant si le tensor contient des NaN
        continue
      test_features.append(tensor)
      test_labels.append(emo)
    #tensors_list = [torch.tensor(dic['markers']) for dic in data_rot]
    #test_features.append(tensors_list)
    #test_labels.append(emo)

    # get audio data
    #x, sr = get_wav(os.path.join(root_path, row['wav_path']))

    # get transcription data
    #header_ph, data_ph = get_ph_fa(os.path.join(root_path, row['FA_ph_path']))




#print(get_labels(annot_file, file_name[:-4]))
#print(vad)
#print(frame_to_s(data_ph[-1]['EFrm']))
#print(x.shape[0]/sr)
#print(data_rot[-1]['time'])
#print(len(data_rot))
#print(len(train_features))
#print(len(test_data))
#print(len(test_labels))
#print(len(train_labels))



2750
[2.5, 2.0, 2.0]
[2.5, 2.0, 2.5]
[2.0, 3.0, 3.0]
[2.0, 3.5, 3.0]
[2.0, 3.5, 2.5]
[2.5, 1.5, 3.0]
[4.0, 2.5, 2.0]
[2.0, 2.0, 2.5]
[2.5, 2.5, 2.5]
[2.5, 1.5, 2.5]
[2.0, 3.5, 3.5]
[2.0, 2.0, 3.0]
[2.5, 2.0, 3.0]
[2.5, 2.5, 3.5]
[2.5, 1.5, 2.5]
[2.5, 1.5, 2.0]
[3.5, 4.5, 2.5]
[3.0, 2.5, 2.5]
[3.0, 2.0, 2.0]
[2.5, 2.0, 2.0]
[3.0, 3.5, 1.5]
[2.5, 2.0, 2.0]
[3.5, 3.0, 2.5]
[3.0, 3.0, 2.0]
[3.5, 3.0, 2.0]
[3.5, 3.0, 2.5]
[4.0, 4.0, 2.5]
[3.5, 3.0, 2.5]
[4.0, 3.0, 2.5]
[3.0, 2.0, 2.5]
[3.0, 3.0, 3.0]
[2.5, 2.0, 1.5]
[3.0, 3.0, 2.5]
[3.0, 3.0, 2.5]
[3.5, 3.0, 3.0]
[3.0, 3.0, 2.5]
[3.5, 3.0, 2.5]
[3.5, 3.0, 2.0]
[3.0, 2.5, 2.0]
[3.5, 2.5, 3.0]
[3.0, 2.5, 3.0]
[3.0, 4.0, 2.5]
[3.5, 3.5, 2.5]
[3.5, 3.5, 2.5]
[3.0, 3.5, 2.5]
[4.0, 3.0, 2.0]
[4.0, 2.0, 1.5]
[3.0, 3.0, 3.0]
[2.5, 2.5, 3.0]
[3.0, 2.0, 2.5]
[3.0, 3.5, 3.5]
[2.0, 4.0, 3.5]
[2.0, 4.5, 4.0]
[3.0, 3.5, 3.0]
[2.0, 2.5, 3.0]
[2.5, 2.5, 3.0]
[2.0, 3.5, 3.5]
[1.5, 4.0, 4.0]
[1.5, 3.5, 4.0]
[1.5, 3.0, 3.5]
[2.0, 3.5, 3.5]
[1.5, 3.5, 4.5]
[3.

In [48]:
import numpy as np
import torch
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import LabelEncoder


# Assuming X_train, y_train, X_test, and y_test as NumPy arrays
X_train = np.array(train_features)
y_train = np.array(train_labels)
X_test = np.array(test_features)
y_test = np.array(test_labels)
#ad one dimension

X_train_tensor = torch.Tensor(np.array(X_train)).unsqueeze(1)
print(X_train_tensor)
X_test_tensor = torch.Tensor(np.array(X_test)).unsqueeze(1)


# Convert string labels to numerical labels using LabelEncoder
label_encoder = LabelEncoder()

# Convert labels to strings
y_train_str = y_train.astype(str)
y_test_str = y_test.astype(str)

y_train_encoded = label_encoder.fit_transform(y_train_str)
y_test_encoded = label_encoder.fit_transform(y_test_str)


y_train_tensor = torch.tensor(y_train_encoded).unsqueeze(1) # Convert to tensor and add a dimension
print(y_train_tensor)
y_test_tensor = torch.tensor(y_test_encoded).unsqueeze(1)    # Convert to tensor and add a dimension


# Convert numerical labels to tensors
y_train_tensor = torch.tensor(y_train_encoded)
y_test_tensor = torch.tensor(y_test_encoded)

# Calculate the number of classes
#num_classes = len(np.unique(y_train_encoded))

# Convert numerical labels to one-hot encoded tensors
#y_train_onehot = F.one_hot(y_train_tensor, num_classes=num_classes).to(torch.long)
#y_test_onehot = F.one_hot(y_test_tensor, num_classes=num_classes).to(torch.long)

# Create one-hot encoded tensors
# Calculate the number of classes
num_classes = len(np.unique(y_train_encoded))

# Convert numerical labels to one-hot encoded tensors
y_train_onehot = torch.zeros(len(y_train_tensor), num_classes)
print("Dimensions de y_train_onehot :", y_train_onehot.shape)
print("Dimensions de y_train_tensor :", y_train_tensor.shape)

index_tensor = torch.arange(y_train_tensor.size(0)).unsqueeze(1)
y_train_onehot.scatter_(1, y_train_tensor.unsqueeze(1), 1)

y_test_onehot = torch.zeros(len(y_test_tensor), num_classes)
y_test_onehot.scatter_(1, y_test_tensor.unsqueeze(1), 1)


#Normalize

X_train_normalized = X_train_tensor
X_test_normalized = X_test_tensor

# Create DataLoader for training and testing with normalized data
train_dataset = TensorDataset(X_train_normalized, y_train_onehot)
train_loader = DataLoader(train_dataset, batch_size = 14, shuffle=True)

test_dataset = TensorDataset(X_test_normalized, y_test_onehot)
test_loader = DataLoader(test_dataset, batch_size = 14, shuffle=False)





tensor([[[-0.0515,  0.0604, -0.0969,  ..., -0.0833,  0.1007,  0.2426]],

        [[-0.0515,  0.0604, -0.0969,  ..., -0.0834,  0.1007,  0.2426]],

        [[-0.0515,  0.0603, -0.0969,  ..., -0.0833,  0.1007,  0.2426]],

        ...,

        [[-0.0532,  0.0723, -0.1036,  ..., -0.0780,  0.1536,  0.2653]],

        [[-0.0532,  0.0727, -0.1042,  ..., -0.0779,  0.1535,  0.2650]],

        [[-0.0531,  0.0738, -0.1053,  ..., -0.0779,  0.1530,  0.2647]]])
tensor([[4],
        [4],
        [4],
        ...,
        [3],
        [3],
        [3]])
Dimensions de y_train_onehot : torch.Size([930791, 6])
Dimensions de y_train_tensor : torch.Size([930791])


In this section we will use the markers

Defining the three models used to train MocapHead/MocapFace/MocapHand  
Inspired from this paper : https://www.researchgate.net/publication/324558351_Multi-Modal_Emotion_recognition_on_IEMOCAP_Dataset_using_Deep_Learning


First Model for MocapHead / MocapHand

In [49]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size=256, output_size=4):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.relu = nn.ReLU()

    def forward(self, x):
        _, (hn, _) = self.lstm(x)
        x = hn[-1, :, :]
        x = self.fc(x)
        x = self.relu(x)
        return x


Second Model for MocapHead/MocapHand

In [54]:
class DenseModel(nn.Module):
    def __init__(self, input_size=165, hidden_size=256, output_size=6):
        super(DenseModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x


Models for MocapFace

First Model

In [39]:
class FacialExprLSTM(nn.Module):
    def __init__(self, input_size, output_size=4):
        super(FacialExprLSTM, self).__init__()
        self.lstm1 = nn.LSTM(input_size, 512, batch_first=True)
        self.lstm2 = nn.LSTM(512, 256, batch_first=True)
        self.fc = nn.Linear(256, output_size)

    def forward(self, x):
        _, (ht1, _) = self.lstm1(x)
        _, (ht2, _) = self.lstm2(ht1)
        out = F.relu(self.fc(ht2[-1]))
        return out


Second Model

In [40]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Model2(nn.Module):
    def __init__(self, input_channels=1, num_classes=4):
        super(Model2, self).__init__()
        self.conv1 = nn.Conv2d(input_channels, 32, kernel_size=3, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=2)
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, stride=2)
        self.conv4 = nn.Conv2d(64, 128, kernel_size=3, stride=2)
        self.conv5 = nn.Conv2d(128, 128, kernel_size=3, stride=2)
        self.dropout = nn.Dropout2d(0.2)
        self.fc1 = nn.Linear(128 * 3 * 3, 256)
        self.fc2 = nn.Linear(256, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = F.relu(self.conv5(x))
        x = self.dropout(x)
        x = x.view(-1, 128 * 3 * 3)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.softmax(x, dim=1)



Third Model

In [41]:
class Model3(nn.Module):
    def __init__(self, input_size=165, num_classes=6):  # Modifier input_size pour correspondre à la taille de vos données
        super(Model3, self).__init__()
        self.fc1 = nn.Linear(input_size, 512)  # Modifier la première couche linéaire
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, num_classes)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return F.softmax(x, dim=1)



In [42]:
class model_FacialMarkers(nn.Module):
    def __init__(self, input_size=165, num_classes=6):  # Modifier input_size pour correspondre à la taille de vos données
        super(model_FacialMarkers, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=2)
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, stride=2)
        self.conv4 = nn.Conv2d(64, 128, kernel_size=3, stride=2)
        self.fc1 = nn.Linear(128 * 3 * 3, 1024)
        self.fc2 = nn.Linear(1024, 256)
        self.fc3 = nn.Linear(256, num_classes)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        # Ajouter une dimension de canal
        x = x.unsqueeze(1)

        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = x.view(-1, 128 * 3 * 3)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return F.softmax(x, dim=1)


Dimensions de mes donnees:

In [50]:
import numpy as np

# Supposons que vous avez des données d'entraînement X_train
print("Dimensions de X_train:", X_train.shape)

# Supposons que vous avez des données de test X_test
print("Dimensions de X_test:", X_test.shape)

# Supposons que vous avez des étiquettes d'entraînement y_train
print("Dimensions de y_train:", y_train.shape)

# Supposons que vous avez des étiquettes de test y_test
print("Dimensions de y_test:", y_test.shape)


Dimensions de X_train: (930791, 165)
Dimensions de X_test: (243447, 165)
Dimensions de y_train: (930791,)
Dimensions de y_test: (243447,)


Join the first model TRAINING LOOP

In [21]:
import torch.nn.functional as F

Debogage

In [44]:
import torch
x = X_train_tensor
# Supposons que x soit votre tensor d'entrée
has_nan = torch.isnan(x).any()

if has_nan:
    print("Le tensor d'entrée contient des valeurs nan.")
else:
    print("Le tensor d'entrée ne contient pas de valeurs nan.")


Le tensor d'entrée ne contient pas de valeurs nan.


Le tensor d'entree contient des valeurs nan !  On remarque que si il existe dans un tensor alors on aura 3 valeurs de NaN car il corresoond a Xi Yi Zi du ieme marquers, alors en gros l'idee c est de remplacer c est trois coordonnees par les coordonnees du barycentre des coordonnees du i-1 et i+1 marqueurs tant qu ils sont eux meme ont des valeurs non NaN , pour assurer une sorte de continuite dans le visage; une premiere proposition simlificatrice .

In [33]:


# Supposons que x soit votre tensor d'entrée de taille [1000, 165]
# Nous supposons que chaque triplet de coordonnées (X, Y, Z) est séparé par une dimension de taille 3

for tensor in x:
  nan_indices = torch.isnan(tensor).any(dim=1)
  for i in range(len(tensor)):
    if nan_indices[i]:
      #trouver les indices des voisins nan
      valid_neighbors_indices = []
      for j in range(-1,2):
        neighbor_index = i+j
        if 0 <= neighbor_index < len(tensor) and not nan_indices[neighbor_index]:
          valid_neighbors_indices.append(neighbor_index)
      if valid_neighbors_indices:
        valid_neighbors = tensor[valid_neighbors_indices]
        barycenter = torch.mean(valid_neighbors,dim=0)
        tensor[i] = barycenter







In [60]:

#Training loop
import torch
import torch.nn as nn
import torch.optim as optim


model = DenseModel()

# Define loss function (criterion) and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Using Adam optimizer with learning rate 0.001

# Training loop
num_epochs = 2

for epoch in range(num_epochs):
    model.train()  # Set the model to train mode

    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    # Iterate over the training dataset
    for inputs, labels in train_loader:
        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        outputs = torch.squeeze(outputs, dim=1)

        #outputs_squeezed = outputs.squeeze(1)
        print(outputs)
        print(labels)
        # Compute the loss
        print("Type de outputs :", outputs.shape)
        print("Type de labels :", labels.shape)

        # Ensure that the batch sizes match
        #batch_size = min(outputs.size(0), labels.size(0))
        #outputs = outputs[:batch_size]
        #labels = labels[:batch_size]

        # Convertir les étiquettes en format flottant
        labels = labels.float()




        loss = criterion(outputs, labels)  # Assuming labels are one-hot encoded

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        # Track the loss
        running_loss += loss.item() * inputs.size(0)

        # Calculate accuracy
        _, predicted = torch.max(outputs, 1)
        total_predictions += labels.size(0)
        correct_predictions += (predicted == torch.argmax(labels, dim=1)).sum().item()

    # Print the average loss and accuracy for this epoch
    epoch_loss = running_loss / len(train_loader.dataset)
    epoch_accuracy = correct_predictions / total_predictions
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy * 100:.2f}%")

print("Training finished")







Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
        [-2.4716e+00, -3.6811e+00,  1.7854e-01, -3.1180e+00, -1.9992e-01,
          3.3836e+00],
        [-7.9143e-01, -3.2185e+00,  2.4823e-01, -3.2691e+00,  4.8342e-01,
          5.8237e-01],
        [ 1.2871e+00, -6.7672e-03,  5.4031e-01, -2.7150e+00, -3.3036e-01,
         -2.4261e+00]], grad_fn=<SqueezeBackward1>)
tensor([[0., 0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 1., 0.],
        [0., 1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 1., 0.],
        [1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1.],
        [0., 0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.]])
Type de outputs : torch.Size([14, 6])
Type de labels : torch.Size([14, 6])
tensor([[-0.8286, -2.7307,  0.4450, -3.6853,  0.45

BERT  model for treating text


In [ ]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels = 5,
    output_attentions = False,
    output_hidden_states = False,
)
print(model)
params = list(model.named_parameters())
optimizer = AdamW(model.parameters(),
                  lr = 2e-5,
                  eps = 1e-8
                )
from transformers import get_linear_schedule_with_warmup

# Number of training epochs. The BERT authors recommend between 2 and 4.
NUM_EPOCHS=4


writer = SummaryWriter(log_dir='/content/')
total_steps = len(train_list) * NUM_EPOCHS

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)
